In [1]:
import pandas as pd
import sqlite3

In [2]:
#Step 1: Load CSV Data
orders_df = pd.read_csv("orders.csv")


In [3]:
#Step 2: Load JSON Data
users_df = pd.read_json("users.json")
users_df.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
#Step 3: Load SQL Data
conn = sqlite3.connect(":memory:")


In [5]:
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)


In [6]:
restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)


In [7]:
print(restaurants_df)

     restaurant_id restaurant_name  cuisine  rating
0                1    Restaurant_1  Chinese     4.8
1                2    Restaurant_2   Indian     4.1
2                3    Restaurant_3  Mexican     4.3
3                4    Restaurant_4  Chinese     4.1
4                5    Restaurant_5  Chinese     4.8
..             ...             ...      ...     ...
495            496  Restaurant_496   Indian     3.1
496            497  Restaurant_497  Mexican     4.4
497            498  Restaurant_498  Chinese     3.9
498            499  Restaurant_499  Mexican     4.9
499            500  Restaurant_500  Chinese     3.2

[500 rows x 4 columns]


In [8]:
#Step 4: Merge the Data
orders_users_df = pd.merge(orders_df,users_df,on="user_id",how="left")



In [9]:
final_df = pd.merge(orders_users_df,restaurants_df,on="restaurant_id",how="left")


In [15]:
#Step 5: Create Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


PermissionError: [Errno 13] Permission denied: 'final_food_delivery_dataset.csv'

In [16]:
print(final_df)

      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                   restaurant_name_x       name       city membership  \
0                  New Foods Chinese  User_2508  Hyderabad    Regular   
1     Ruchi Curry House Multicuisine 

In [17]:
#Order trends over time
final_df.groupby("order_date")["order_id"].count()


order_date
01-01-2023    29
01-01-2024    27
01-02-2023    25
01-03-2023    37
01-04-2023    32
              ..
31-05-2023    33
31-07-2023    27
31-08-2023    24
31-10-2023    30
31-12-2023    29
Name: order_id, Length: 366, dtype: int64

In [18]:
#User behavior patterns
final_df.groupby("membership")["order_id"].count()


membership
Gold       4987
Regular    5013
Name: order_id, dtype: int64

In [19]:
#City-wise and cuisine-wise performance
final_df.groupby(["city", "cuisine"])["total_amount"].sum()


city       cuisine
Bangalore  Chinese    525046.13
           Indian     543014.04
           Italian    567881.80
           Mexican    571004.61
Chennai    Chinese    491710.42
           Indian     441946.13
           Italian    522051.88
           Mexican    534804.60
Hyderabad  Chinese    470061.40
           Indian     467210.85
           Italian    452008.82
           Mexican    500085.51
Pune       Chinese    443686.70
           Indian     519241.56
           Italian    482261.30
           Mexican    479608.37
Name: total_amount, dtype: float64

In [20]:
#Membership impact (Gold vs Regular)
final_df.groupby("membership")["total_amount"].mean()


membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64

In [21]:
#Revenue distribution and seasonality
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df.groupby(final_df["order_date"].dt.month)["total_amount"].sum()


C:\Users\gaidh\AppData\Local\Temp\ipykernel_444\2299128527.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


order_date
1     663020.26
2     630867.40
3     716738.98
4     666097.18
5     668428.61
6     610822.93
7     688559.45
8     694987.58
9     653838.07
10    699187.73
11    637772.43
12    681303.50
Name: total_amount, dtype: float64

In [22]:
#MCQ

In [23]:
df = pd.read_csv("final_food_delivery_dataset.csv")


In [24]:
conn = sqlite3.connect(":memory:")


In [25]:
df.to_sql(name="final_food_delivery_dataset",con=conn,index=False,if_exists="replace")



10000

In [26]:
pd.read_sql_query("SELECT * FROM final_food_delivery_dataset LIMIT 5;",conn)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [27]:
#Which city has the highest total revenue (total_amount) from Gold members?  
query = """
SELECT city, SUM(total_amount) AS total_revenue
FROM final_food_delivery_dataset
WHERE membership = 'Gold'
GROUP BY city
ORDER BY total_revenue DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)


        city  total_revenue
0    Chennai     1080909.79
1       Pune     1003012.32
2  Bangalore      994702.59
3  Hyderabad      896740.19


In [28]:
# Which cuisine has the highest average order value?
query = """
SELECT cuisine, AVG(total_amount) AS avg_order_value
FROM final_food_delivery_dataset
GROUP BY cuisine
ORDER BY avg_order_value DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)


   cuisine  avg_order_value
0  Mexican       808.021344
1  Italian       799.448578
2   Indian       798.466011
3  Chinese       798.389020


In [29]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
query = """
SELECT COUNT(*) AS user_count
FROM (SELECT user_id, SUM(total_amount) AS total_spent
    FROM final_food_delivery_dataset
    GROUP BY user_id
    HAVING total_spent > 1000);
"""
result = pd.read_sql_query(query, conn)
print(result)


   user_count
0        2544


In [30]:
#Which restaurant rating range generated the highest total revenue?
query = """
SELECT rating_range, SUM(total_amount) AS total_revenue
FROM (
    SELECT total_amount,
           CASE
               WHEN rating BETWEEN 3.0 AND 3.5 THEN '3.0–3.5'
               WHEN rating BETWEEN 3.6 AND 4.0 THEN '3.6–4.0'
               WHEN rating BETWEEN 4.1 AND 4.5 THEN '4.1–4.5'
               WHEN rating BETWEEN 4.6 AND 5.0 THEN '4.6–5.0'
           END AS rating_range
    FROM final_food_delivery_dataset
)
GROUP BY rating_range
ORDER BY total_revenue DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)


  rating_range  total_revenue
0      4.6–5.0     2197030.75
1      3.0–3.5     2136772.70
2      4.1–4.5     1960326.26
3      3.6–4.0     1717494.41


In [31]:
#Among Gold members, which city has the highest average order value?
query = """
SELECT city, AVG(total_amount) AS avg_order_value
FROM final_food_delivery_dataset
WHERE membership = 'Gold'
GROUP BY city
ORDER BY avg_order_value DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)


        city  avg_order_value
0    Chennai       808.459080
1  Hyderabad       806.421034
2  Bangalore       793.223756
3       Pune       781.162243


In [32]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
query = """
SELECT cuisine,
       COUNT(DISTINCT restaurant_name_x) AS restaurant_count,
       SUM(total_amount) AS total_revenue
FROM final_food_delivery_dataset
GROUP BY cuisine
ORDER BY restaurant_count ASC, total_revenue DESC;
"""

result = pd.read_sql_query(query, conn)
print(result)


   cuisine  restaurant_count  total_revenue
0  Chinese               115     1930504.65
1   Indian               122     1971412.58
2  Mexican               124     2085503.09
3  Italian               124     2024203.80


In [33]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

query = """
SELECT ROUND(
    (SUM(CASE WHEN membership = 'Gold' THEN 1 ELSE 0 END) * 100.0)/ COUNT(*)
) AS gold_order_percentage
FROM final_food_delivery_dataset;
"""
result = pd.read_sql_query(query, conn)
print(result)

   gold_order_percentage
0                   50.0


In [34]:
#Which restaurant has the highest average order value but less than 20 total orders?
query = """
SELECT restaurant_name_x,
       AVG(total_amount) AS avg_order_value,
       COUNT(*) AS total_orders
FROM final_food_delivery_dataset
GROUP BY restaurant_name_x
HAVING total_orders < 20
ORDER BY avg_order_value DESC;
"""
result = pd.read_sql_query(query, conn)
result.head(20)

,restaurant_name_x,avg_order_value,total_orders
0,Hotel Dhaba Multicuisine,1040.222308,13
1,Sri Mess Punjabi,1029.180833,12
2,Ruchi Biryani Punjabi,1002.140625,16
3,Sri Delights Pure Veg,989.467222,18
4,Classic Kitchen Family Restaurant,973.167895,19
5,Hotel Dhaba Chinese,973.125556,18
6,Amma Mess Pure Veg,965.299444,18
7,Hotel Biryani Pure Veg,964.577692,13
8,Annapurna Curry House Multicuisine,954.512353,17
9,Taste of Restaurant Family Restaurant,948.220000,10


In [35]:
#Which combination contributes the highest revenue?
query = """
SELECT membership, cuisine, SUM(total_amount) AS total_revenue
FROM final_food_delivery_dataset
GROUP BY membership, cuisine
ORDER BY total_revenue DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)

  membership  cuisine  total_revenue
0    Regular  Mexican     1072943.30
1    Regular  Italian     1018424.75
2       Gold  Mexican     1012559.79
3       Gold  Italian     1005779.05
4    Regular   Indian      992100.27
5       Gold   Indian      979312.31
6       Gold  Chinese      977713.74
7    Regular  Chinese      952790.91


In [36]:
#During which quarter of the year is the total revenue highest?

query = """
SELECT quarter, SUM(total_amount) AS total_revenue
FROM (
    SELECT total_amount,
           CASE
               WHEN strftime('%m', order_date) IN ('01','02','03') THEN 'Q1'
               WHEN strftime('%m', order_date) IN ('04','05','06') THEN 'Q2'
               WHEN strftime('%m', order_date) IN ('07','08','09') THEN 'Q3'
               ELSE 'Q4'
           END AS quarter
    FROM final_food_delivery_dataset
)
GROUP BY quarter
ORDER BY total_revenue DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)

  quarter  total_revenue
0      Q4     8011624.12


In [37]:
#How many total orders were placed by users with Gold membership?
query = """
SELECT COUNT(*) AS total_gold_orders
FROM final_food_delivery_dataset
WHERE membership = 'Gold';
"""

pd.read_sql_query(query, conn)


,total_gold_orders
0,4987


In [38]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
query = """
SELECT ROUND(SUM(total_amount)) AS total_revenue
FROM final_food_delivery_dataset
WHERE city = 'Hyderabad';
"""
pd.read_sql_query(query, conn)


,total_revenue
0,1889367.0


In [39]:
#How many distinct users placed at least one order?
query = """
SELECT COUNT(DISTINCT user_id) AS total_users
FROM final_food_delivery_dataset;
"""
pd.read_sql_query(query, conn)


,total_users
0,2883


In [40]:
#What is the average order value (rounded to 2 decimals) for Gold members?
query = """
SELECT ROUND(AVG(total_amount), 2) AS avg_order_value
FROM final_food_delivery_dataset
WHERE membership = 'Gold';
"""
pd.read_sql_query(query, conn)


,avg_order_value
0,797.15


In [41]:
#How many orders were placed for restaurants with rating ≥ 4.5?
query = """
SELECT COUNT(*) AS high_rating_orders
FROM final_food_delivery_dataset
WHERE rating >= 4.5;
"""
pd.read_sql_query(query, conn)


,high_rating_orders
0,3374


In [42]:
#How many orders were placed in the top revenue city among Gold members only?
query = """
SELECT COUNT(*) AS total_orders
FROM final_food_delivery_dataset
WHERE membership = 'Gold'
AND city = (
    SELECT city
    FROM final_food_delivery_dataset
    WHERE membership = 'Gold'
    GROUP BY city
    ORDER BY SUM(total_amount) DESC
    LIMIT 1);
"""
pd.read_sql_query(query, conn)


,total_orders
0,1337
